In [54]:
import pandas as pd
import numpy as np
import scipy

In [61]:
fp = '../results/06242021_new_results_format/som_germ_cnv_pur_target_phospho/coef_results.txt'
out = fp.replace('coef_results.txt', 'driver_scores_p05.txt')

In [62]:
df = pd.read_csv(fp, sep='\t')
df

,coef,p-value,FDR,-log10(FDR),feature,driver,target,disease,model_r2,model_r2_FDR
0,-0.016586,5.696457e-01,7.578885e-01,0.120395,driver_gene_cnv,ABL1,ABI1-204|ABI1|452_174_187_1_0,BR,0.020068,3.566061e-01
1,-0.061267,3.770408e-02,1.026869e-01,0.988485,TumorPurity,ABL1,ABI1-204|ABI1|452_174_187_1_0,BR,0.020068,3.566061e-01
2,-0.234853,3.007376e-01,5.128425e-01,0.290016,driver_gene_has_nonsilent_mutation,ABL1,ABI1-204|ABI1|452_174_187_1_0,BR,0.020068,3.566061e-01
3,0.000000,NaN,NaN,NaN,driver_gene_is_pathogenic_germline,ABL1,ABI1-204|ABI1|452_174_187_1_0,BR,0.020068,3.566061e-01
4,-0.024269,3.489810e-01,5.651451e-01,0.247840,driver_gene_cnv,ABL1,ABI1-204|ABI1|452_174_187_1_1_S183,BR,0.018621,3.711768e-01
...,...,...,...,...,...,...,...,...,...,...
3518311,0.000000,NaN,NaN,NaN,driver_gene_is_pathogenic_germline,ZNF750,ZNF777-201|ZNF777|831_604_610_1_0,ccRCC,0.041873,5.061344e-04
3518312,0.125712,8.089208e-21,2.521268e-20,19.598381,TumorPurity,ZNF750,ZNF839-209|ZNF839|811_670_677_1_0,ccRCC,0.246058,1.064838e-21
3518313,-0.019994,1.701699e-01,2.168608e-01,0.663819,driver_gene_cnv,ZNF750,ZNF839-209|ZNF839|811_670_677_1_0,ccRCC,0.246058,1.064838e-21
3518314,0.515714,7.632312e-04,1.355022e-03,2.868054,driver_gene_has_nonsilent_mutation,ZNF750,ZNF839-209|ZNF839|811_670_677_1_0,ccRCC,0.246058,1.064838e-21


N = number of protein pairs/rows for given driver

C = coef

one version with all, and one with p < .05

In [63]:
def calculate_s2(n, cs):
    num = np.sum(np.square(cs))
    num = np.sqrt(num / n)
    return num

def calculate_s4(ps, n, s2):
    return -np.log10(np.sum(ps) / n) * s2

def calculate_s5(ps, n, s2):
    return -np.log10(n / np.sum(1 / ps)) * s2

def calculate_scores(df, driver, disease=None, feature=None, fdr=None):
    filtered = df[df['driver']==driver]
    filtered = filtered[['coef', 'p-value', 'feature', 'driver', 'target', 'disease', 'model_r2_FDR']]
    filtered = filtered[np.sum(pd.isnull(filtered.values), axis=1).flatten()==0]
    if fdr is not None:
        filtered = filtered[filtered['model_r2_FDR']<=fdr]
    if disease is not None:
        filtered = filtered[filtered['disease']==disease]
    if feature is not None:
        filtered = filtered[filtered['feature']==feature]

    n = filtered.shape[0]
    
    cs = filtered['coef'].to_numpy()
    ps = filtered['p-value'].to_numpy()
    
    s2 = calculate_s2(n, cs)
    s4 = calculate_s4(ps, n, s2)
    s5 = calculate_s5(ps, n, s2)
    
    return s2, s4, s5
    
    
    
    

In [64]:
data = []
fdr = .05
# fdr = None
drivers = sorted(set(df['driver']))
types = sorted(set(df['feature']))
for driver in drivers:
    s2, s4, s5 = calculate_scores(df, driver, fdr=fdr)
    data.append([driver, 'pancan', 'all', s2, s4, s5])
    for disease in sorted(set(df['disease'])):
        s2, s4, s5 = calculate_scores(df, driver, disease=disease, fdr=fdr)
        data.append([driver, disease, 'all', s2, s4, s5])
        for t in types:
            s2, s4, s5 = calculate_scores(df, driver, disease=disease, feature=t, fdr=fdr)
            data.append([driver, disease, t, s2, s4, s5])
results = pd.DataFrame(data=data, columns=['driver', 'disease', 'feature', 's2', 's4', 's5'])
results

/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3

/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3

/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3

/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3

/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3

...

/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3

/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3

/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3

/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/diskmnt/Projects/Users/estorrs/miniconda3/envs/cptac_driver/lib/python3

,driver,disease,feature,s2,s4,s5
0,ABL1,pancan,all,0.355095,0.254562,29.087690
1,ABL1,BR,all,0.453933,0.288591,3.255828
2,ABL1,BR,TumorPurity,0.194639,0.278740,1.488911
3,ABL1,BR,driver_gene_cnv,0.096162,0.043990,0.274867
4,ABL1,BR,driver_gene_has_nonsilent_mutation,0.755669,0.386208,2.248037
...,...,...,...,...,...,...
8512,ZNF750,ccRCC,all,0.269894,0.191135,7.468103
8513,ZNF750,ccRCC,TumorPurity,0.175095,0.283725,4.928514
8514,ZNF750,ccRCC,driver_gene_cnv,0.066667,0.044643,0.673595
8515,ZNF750,ccRCC,driver_gene_has_nonsilent_mutation,0.428282,0.195555,1.506042


In [65]:
results.to_csv(out, sep='\t', index=False)

In [66]:
results

,driver,disease,feature,s2,s4,s5
0,ABL1,pancan,all,0.355095,0.254562,29.087690
1,ABL1,BR,all,0.453933,0.288591,3.255828
2,ABL1,BR,TumorPurity,0.194639,0.278740,1.488911
3,ABL1,BR,driver_gene_cnv,0.096162,0.043990,0.274867
4,ABL1,BR,driver_gene_has_nonsilent_mutation,0.755669,0.386208,2.248037
...,...,...,...,...,...,...
8512,ZNF750,ccRCC,all,0.269894,0.191135,7.468103
8513,ZNF750,ccRCC,TumorPurity,0.175095,0.283725,4.928514
8514,ZNF750,ccRCC,driver_gene_cnv,0.066667,0.044643,0.673595
8515,ZNF750,ccRCC,driver_gene_has_nonsilent_mutation,0.428282,0.195555,1.506042
